In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer,RegexTokenizer

In [4]:
from pyspark.sql.functions import col,udf
from pyspark.sql.types import IntegerType

In [5]:
df = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
],['id','sentence'])

In [6]:
df.show()

+---+--------------------+
 id| sentence|
+---+--------------------+
 0|Hi I heard about ...|
 1|I wish Java could...|
 2|Logistic,regressi...|
+---+--------------------+

In [7]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')
regex_tokenizer = RegexTokenizer(inputCol='sentence',outputCol='words',pattern='\\W')

In [8]:
count_tokens = udf(lambda words:len(words),IntegerType())

In [9]:
tokenized = tokenizer.transform(df)

In [10]:
tokenized.withColumn('count',count_tokens(col('words'))).show()

+---+--------------------+--------------------+-----+
 id| sentence| words|count|
+---+--------------------+--------------------+-----+
 0|Hi I heard about ...|[hi, i, heard, ab...| 5|
 1|I wish Java could...|[i, wish, java, c...| 7|
 2|Logistic,regressi...|[logistic,regress...| 1|
+---+--------------------+--------------------+-----+

In [11]:
rg_tokenized = regex_tokenizer.transform(df)

In [12]:
rg_tokenized.withColumn('counts',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
 id| sentence| words|counts|
+---+--------------------+--------------------+------+
 0|Hi I heard about ...|[hi, i, heard, ab...| 5|
 1|I wish Java could...|[i, wish, java, c...| 7|
 2|Logistic,regressi...|[logistic, regres...| 5|
+---+--------------------+--------------------+------+

In [13]:
from pyspark.ml.feature import StopWordsRemover

In [14]:
sentenceData = spark.createDataFrame([
    (0, ["I", "saw", "the", "red", "balloon"]),
    (1, ["Mary", "had", "a", "little", "lamb"])
], ["id", "tokens"])

In [15]:
sentenceData.show()

+---+--------------------+
 id| tokens|
+---+--------------------+
 0|[I, saw, the, red...|
 1|[Mary, had, a, li...|
+---+--------------------+

In [16]:
remover = StopWordsRemover(inputCol='tokens',outputCol='filtered')
remover.transform(sentenceData).show()

+---+--------------------+--------------------+
 id| tokens| filtered|
+---+--------------------+--------------------+
 0|[I, saw, the, red...| [saw, red, balloon]|
 1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+

In [17]:
from pyspark.ml.feature import NGram

In [18]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

In [19]:
ngram = NGram(n=2,inputCol='words',outputCol='grams')

In [20]:
ngram.transform(wordDataFrame).show(truncate=False)

+---+------------------------------------------+------------------------------------------------------------------+
id |words |grams |
+---+------------------------------------------+------------------------------------------------------------------+
0 |[Hi, I, heard, about, Spark] |[Hi I, I heard, heard about, about Spark] |
1 |[I, wish, Java, could, use, case, classes]|[I wish, wish Java, Java could, could use, use case, case classes]|
2 |[Logistic, regression, models, are, neat] |[Logistic regression, regression models, models are, are neat] |
+---+------------------------------------------+------------------------------------------------------------------+

In [21]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [22]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

In [23]:
sentenceData.show()

+-----+--------------------+
label| sentence|
+-----+--------------------+
 0.0|Hi I heard about ...|
 0.0|I wish Java could...|
 1.0|Logistic regressi...|
+-----+--------------------+

In [24]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')

In [25]:
words_data = tokenizer.transform(sentenceData)

In [26]:
words_data.show()

+-----+--------------------+--------------------+
label| sentence| words|
+-----+--------------------+--------------------+
 0.0|Hi I heard about ...|[hi, i, heard, ab...|
 0.0|I wish Java could...|[i, wish, java, c...|
 1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+

In [27]:
hashing_tf = HashingTF(inputCol='words',outputCol='raw_features')

In [28]:
featurized_data = hashing_tf.transform(words_data)

In [29]:
idf = IDF(inputCol="raw_features", outputCol="features")

In [30]:
idf_model = idf.fit(featurized_data)

In [31]:
rescaled_data = idf_model.transform(featurized_data)

In [32]:
rescaled_data.show()

+-----+--------------------+--------------------+--------------------+--------------------+
label| sentence| words| raw_features| features|
+-----+--------------------+--------------------+--------------------+--------------------+
 0.0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[24417,49...|(262144,[24417,49...|
 0.0|I wish Java could...|[i, wish, java, c...|(262144,[20719,24...|(262144,[20719,24...|
 1.0|Logistic regressi...|[logistic, regres...|(262144,[13671,91...|(262144,[13671,91...|
+-----+--------------------+--------------------+--------------------+--------------------+

In [33]:
from pyspark.ml.feature import CountVectorizer

In [34]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [35]:
count_vec = CountVectorizer(inputCol='words',outputCol='features',vocabSize=3)

In [36]:
model = count_vec.fit(df)

In [37]:
result = model.transform(df)

In [38]:
result.show()

+---+---------------+--------------------+
 id| words| features|
+---+---------------+--------------------+
 0| [a, b, c]|(3,[0,1,2],[1.0,1...|
 1|[a, b, b, c, a]|(3,[0,1,2],[2.0,2...|
+---+---------------+--------------------+